In [1]:
import pandas as pd
from imblearn.over_sampling import SMOTE

In [2]:
df = pd.read_excel(r'C:\Users\Lenovo\Desktop\ML_health\Anxiety_ML_ITEMS.xlsx')
df.head()

,Number,Residence,Hukou,Estates,Debt,Income,Gender,Age,Education,Marriage,...,WAFCS8,WAFCS9,WAFCS10,Chronic1,Chronic2,Chronic3,Chronic4,Chronic5,Chronic6,Anxiety
0,1,2,2,2,1,7,2,1,6,0,...,-1.03774,-1.04425,-1.04324,-0.22618,0.53747,-0.55702,-0.81609,-0.65780,0.47164,0
1,2,2,2,1,0,0,2,1,6,0,...,-1.03774,-1.04425,-1.04324,-0.97408,-0.23181,-0.55702,-0.04902,-0.65780,-0.99348,0
2,3,2,2,3,1,1,2,4,3,1,...,-0.33977,-0.29348,-0.29708,0.52172,-1.00109,-0.55702,-0.81609,-0.65780,-0.26092,0
3,4,1,1,1,0,2,2,0,2,0,...,-1.03774,-1.04425,-1.04324,0.52172,-0.23181,1.29179,0.71804,1.01182,0.47164,0
4,5,2,2,0,0,0,1,1,6,0,...,-1.03774,-1.04425,-1.04324,1.26962,-0.23181,2.21620,0.71804,1.84663,1.20420,1


In [3]:
X = df.drop('Anxiety', axis = 1)
X.shape

(11031, 112)

In [4]:
y=df['Anxiety']
y.shape

(11031,)

In [5]:
pd.Series(y).value_counts()

Anxiety
0    9534
1    1497
Name: count, dtype: int64

In [6]:
smote = SMOTE()

In [7]:
X.dtypes

Number         int64
Residence      int64
Hukou          int64
Estates        int64
Debt           int64
              ...   
Chronic2     float64
Chronic3     float64
Chronic4     float64
Chronic5     float64
Chronic6     float64
Length: 112, dtype: object

In [8]:
X = X.apply(pd.to_numeric, errors='coerce')

In [9]:
X.dtypes

Number         int64
Residence      int64
Hukou          int64
Estates        int64
Debt           int64
              ...   
Chronic2     float64
Chronic3     float64
Chronic4     float64
Chronic5     float64
Chronic6     float64
Length: 112, dtype: object

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [11]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [12]:
print(y_train.value_counts())  # 原始训练集的类别分布
print(y_train_resampled.value_counts())  # 过采样后的类别分布

Anxiety
0    7647
1    1177
Name: count, dtype: int64
Anxiety
1    7647
0    7647
Name: count, dtype: int64


In [13]:
df2=pd.concat([y_train_resampled, X_train_resampled], axis=1)

In [14]:
df2.to_excel(r'C:\Users\Lenovo\Desktop\ML_health\Anxiety_ML_ITEMS_smote.xlsx',index = False)

In [15]:
df3 = pd.read_excel(r'C:\Users\Lenovo\Desktop\ML_health\Anxiety_ML_ITEMS_smote.xlsx')

In [16]:
X = df3.drop('Anxiety', axis = 1)
X.shape

(15294, 112)

In [17]:
y=df3['Anxiety']
y.shape

(15294,)

In [18]:
X = df3.drop(['Anxiety', 'Number'], axis=1)

In [19]:
X.shape

(15294, 111)

In [20]:
y=df3['Anxiety']
y.shape

(15294,)

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [22]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB()

In [23]:
# 使用训练好的模型对验证集进行预测
y_pred = model.predict(X_val)

# 验证集的真实标签
y_true = y_val

In [24]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 打印分类报告
print(classification_report(y_true, y_pred))

# 打印混淆矩阵
conf_mat = confusion_matrix(y_true, y_pred)
print(conf_mat)

# 计算并打印准确率
print("Accuracy:", accuracy_score(y_true, y_pred))

# 计算其他性能指标
print("Precision:", precision_score(y_true, y_pred, average='macro'))
print("Recall:", recall_score(y_true, y_pred, average='macro'))
print("F1 Score:", f1_score(y_true, y_pred, average='macro'))

# 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
y_scores = model.predict_proba(X_val)[:, 1]  # 获取正类的概率
print("ROC AUC:", roc_auc_score(y_true, y_scores))

# 计算Specificity
tn, fp, fn, tp = conf_mat.ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)

              precision    recall  f1-score   support

           0       0.84      0.82      0.83      1564
           1       0.82      0.84      0.83      1495

    accuracy                           0.83      3059
   macro avg       0.83      0.83      0.83      3059
weighted avg       0.83      0.83      0.83      3059

[[1285  279]
 [ 240 1255]]
Accuracy: 0.8303367113435763
Precision: 0.8303727531151815
Recall: 0.8305380680700374
F1 Score: 0.8303203916045829
ROC AUC: 0.9052932195126123
Specificity: 0.821611253196931


In [25]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [26]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [27]:
# 使用训练好的模型对验证集进行预测
y_pred = model.predict(X_val)

# 验证集的真实标签
y_true = y_val

In [28]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 打印分类报告
print(classification_report(y_true, y_pred))

# 打印混淆矩阵
conf_mat = confusion_matrix(y_true, y_pred)
print(conf_mat)

# 计算并打印准确率
print("Accuracy:", accuracy_score(y_true, y_pred))

# 计算其他性能指标
print("Precision:", precision_score(y_true, y_pred, average='macro'))
print("Recall:", recall_score(y_true, y_pred, average='macro'))
print("F1 Score:", f1_score(y_true, y_pred, average='macro'))

# 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
y_scores = model.predict_proba(X_val)[:, 1]  # 获取正类的概率
print("ROC AUC:", roc_auc_score(y_true, y_scores))

# 计算Specificity
tn, fp, fn, tp = conf_mat.ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)

              precision    recall  f1-score   support

           0       0.92      0.91      0.91      1564
           1       0.91      0.92      0.91      1495

    accuracy                           0.91      3059
   macro avg       0.91      0.91      0.91      3059
weighted avg       0.91      0.91      0.91      3059

[[1424  140]
 [ 126 1369]]
Accuracy: 0.9130434782608695
Precision: 0.9129665020628914
Recall: 0.9131024985244934
F1 Score: 0.9130153586697078
ROC AUC: 0.9131024985244934
Specificity: 0.9104859335038363


In [29]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [30]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
model.fit(X_train, y_train)

KNeighborsClassifier()

In [31]:
# 使用训练好的模型对验证集进行预测
y_pred = model.predict(X_val)

# 验证集的真实标签
y_true = y_val

In [32]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 打印分类报告
print(classification_report(y_true, y_pred))

# 打印混淆矩阵
conf_mat = confusion_matrix(y_true, y_pred)
print(conf_mat)

# 计算并打印准确率
print("Accuracy:", accuracy_score(y_true, y_pred))

# 计算其他性能指标
print("Precision:", precision_score(y_true, y_pred, average='macro'))
print("Recall:", recall_score(y_true, y_pred, average='macro'))
print("F1 Score:", f1_score(y_true, y_pred, average='macro'))

# 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
y_scores = model.predict_proba(X_val)[:, 1]  # 获取正类的概率
print("ROC AUC:", roc_auc_score(y_true, y_scores))

# 计算Specificity
tn, fp, fn, tp = conf_mat.ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)

              precision    recall  f1-score   support

           0       0.99      0.84      0.90      1564
           1       0.85      0.99      0.91      1495

    accuracy                           0.91      3059
   macro avg       0.92      0.91      0.91      3059
weighted avg       0.92      0.91      0.91      3059

[[1306  258]
 [  18 1477]]
Accuracy: 0.9097744360902256
Precision: 0.9188508319040197
Recall: 0.9114991146960456
F1 Score: 0.9094916082776601
ROC AUC: 0.9718969454875159
Specificity: 0.8350383631713555


In [33]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [34]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [35]:
# 使用训练好的模型对验证集进行预测
y_pred = model.predict(X_val)

# 验证集的真实标签
y_true = y_val

In [36]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 打印分类报告
print(classification_report(y_true, y_pred))

# 打印混淆矩阵
conf_mat = confusion_matrix(y_true, y_pred)
print(conf_mat)

# 计算并打印准确率
print("Accuracy:", accuracy_score(y_true, y_pred))

# 计算其他性能指标
print("Precision:", precision_score(y_true, y_pred, average='macro'))
print("Recall:", recall_score(y_true, y_pred, average='macro'))
print("F1 Score:", f1_score(y_true, y_pred, average='macro'))

# 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
y_scores = model.predict_proba(X_val)[:, 1]  # 获取正类的概率
print("ROC AUC:", roc_auc_score(y_true, y_scores))

# 计算Specificity
tn, fp, fn, tp = conf_mat.ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)

              precision    recall  f1-score   support

           0       0.88      0.87      0.88      1564
           1       0.87      0.88      0.87      1495

    accuracy                           0.88      3059
   macro avg       0.88      0.88      0.88      3059
weighted avg       0.88      0.88      0.88      3059

[[1367  197]
 [ 185 1310]]
Accuracy: 0.8751225890813992
Precision: 0.875037838882466
Recall: 0.8751475506590596
F1 Score: 0.8750792154823613
ROC AUC: 0.9475472375950527
Specificity: 0.8740409207161125


In [50]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [51]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

In [52]:
param_grid = {
    'n_estimators': [500],
    'learning_rate': [0.1],
    'num_leaves': [51]
}

# 创建 LightGBM 分类器
gbm = LGBMClassifier()

# 创建网格搜索对象
grid_search = GridSearchCV(estimator=gbm, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1)

# 执行网格搜索
grid_search.fit(X_train, y_train)

# 最佳参数和最佳模型
print("Best parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[LightGBM] [Info] Number of positive: 4921, number of negative: 4867
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015355 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 24315
[LightGBM] [Info] Number of data points in the train set: 9788, number of used features: 111
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502758 -> initscore=0.011034
[LightGBM] [Info] Start training from score 0.011034
[LightGBM] [Info] Number of positive: 4921, number of negative: 4867
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006302 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 24315
[LightGBM] [Info] Number of data points in the train set: 9788, number of used features: 111
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502758 -> initscore=0.011034
[LightGBM] [

In [53]:
y_pred = best_model.predict(X_val)
# 验证集的真实标签
y_true = y_val

In [54]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# 打印分类报告
print(classification_report(y_true, y_pred))

# 打印混淆矩阵
print(confusion_matrix(y_true, y_pred))

# 计算并打印准确率
print("Accuracy:", accuracy_score(y_true, y_pred))

# 计算其他性能指标
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

print("Precision:", precision_score(y_true, y_pred, average='macro'))
print("Recall:", recall_score(y_true, y_pred, average='macro'))
print("F1 Score:", f1_score(y_true, y_pred, average='macro'))

# 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
y_scores = best_model.predict_proba(X_val)[:, 1]  # 获取正类的概率
print("ROC AUC:", roc_auc_score(y_true, y_scores))

# 计算Specificity
tn, fp, fn, tp = conf_mat.ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)

              precision    recall  f1-score   support

           0       0.93      0.98      0.95      1564
           1       0.97      0.92      0.95      1495

    accuracy                           0.95      3059
   macro avg       0.95      0.95      0.95      3059
weighted avg       0.95      0.95      0.95      3059

[[1525   39]
 [ 113 1382]]
Accuracy: 0.9503105590062112
Precision: 0.9517839850352166
Recall: 0.9497393271689947
F1 Score: 0.9502017343412767
ROC AUC: 0.9888160877263513
Specificity: 0.8740409207161125


In [55]:
# 获取特征重要性
feature_importances = best_model.feature_importances_

# 排序特征
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 打印特征重要性排序
print(importance_df)
print(importance_df.head(10))

            Feature  Importance
16             ZBMI        1747
72        EQ5D5L_VS        1203
4            Income         617
6               Age         580
94          Stress3         546
..              ...         ...
66  MedCompliance10          36
63   MedCompliance7          30
59   MedCompliance3          27
64   MedCompliance8          26
58   MedCompliance2          23

[111 rows x 2 columns]
       Feature  Importance
16        ZBMI        1747
72   EQ5D5L_VS        1203
4       Income         617
6          Age         580
94     Stress3         546
93     Stress2         546
92     Stress1         544
107   Chronic3         505
7    Education         445
106   Chronic2         405


In [56]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [57]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [58]:
# 使用训练好的模型对验证集进行预测
y_pred = model.predict(X_val)

# 验证集的真实标签
y_true = y_val

In [59]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# 打印分类报告
print(classification_report(y_true, y_pred))

# 打印混淆矩阵
print(confusion_matrix(y_true, y_pred))

# 计算并打印准确率
print("Accuracy:", accuracy_score(y_true, y_pred))

# 计算其他性能指标
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

print("Precision:", precision_score(y_true, y_pred, average='macro'))
print("Recall:", recall_score(y_true, y_pred, average='macro'))
print("F1 Score:", f1_score(y_true, y_pred, average='macro'))

# 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
y_scores = model.predict_proba(X_val)[:, 1]  # 获取正类的概率
print("ROC AUC:", roc_auc_score(y_true, y_scores))

# 计算Specificity
tn, fp, fn, tp = conf_mat.ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)

              precision    recall  f1-score   support

           0       0.94      0.97      0.95      1564
           1       0.97      0.94      0.95      1495

    accuracy                           0.95      3059
   macro avg       0.95      0.95      0.95      3059
weighted avg       0.95      0.95      0.95      3059

[[1515   49]
 [  95 1400]]
Accuracy: 0.9529257927427264
Precision: 0.9535886818495514
Recall: 0.9525624631123353
F1 Score: 0.952859168241966
ROC AUC: 0.990187881172536
Specificity: 0.8740409207161125


In [60]:
# 获取特征重要性
feature_importances = model.feature_importances_

# 排序特征
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 打印特征重要性排序
print(importance_df)
print(importance_df.head(10))

             Feature  Importance
109         Chronic5    0.085598
107         Chronic3    0.070102
93           Stress2    0.062501
91           EBS_SF7    0.049750
92           Stress1    0.046854
..               ...         ...
62    MedCompliance6    0.001440
66   MedCompliance10    0.001436
59    MedCompliance3    0.001327
58    MedCompliance2    0.001249
0          Residence    0.001140

[111 rows x 2 columns]
       Feature  Importance
109   Chronic5    0.085598
107   Chronic3    0.070102
93     Stress2    0.062501
91     EBS_SF7    0.049750
92     Stress1    0.046854
94     Stress3    0.037916
52     FHS_SF6    0.031008
108   Chronic4    0.028312
71   EQ_5D_5L5    0.027081
90     EBS_SF6    0.026679


In [61]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [62]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
model = RandomForestClassifier()
model.fit(X_train, y_train)
# 定义参数网格
param_grid = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# 设置网格搜索
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1)

# 执行网格搜索
grid_search.fit(X_train, y_train)

# 输出最佳参数
print("Best parameters:", grid_search.best_params_)

# 获取最佳模型
best_rf = grid_search.best_estimator_

Fitting 5 folds for each of 144 candidates, totalling 720 fits


D:\anaconda\envs\MentalAssessment\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best parameters: {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


In [63]:
y_pred = best_rf.predict(X_val)
# 验证集的真实标签
y_true = y_val

In [64]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 打印分类报告
print(classification_report(y_true, y_pred))

# 打印混淆矩阵
conf_mat = confusion_matrix(y_true, y_pred)
print(conf_mat)

# 计算并打印准确率
print("Accuracy:", accuracy_score(y_true, y_pred))

# 计算其他性能指标
print("Precision:", precision_score(y_true, y_pred, average='macro'))
print("Recall:", recall_score(y_true, y_pred, average='macro'))
print("F1 Score:", f1_score(y_true, y_pred, average='macro'))

# 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
y_scores = best_rf.predict_proba(X_val)[:, 1]  # 获取正类的概率
print("ROC AUC:", roc_auc_score(y_true, y_scores))

# 计算Specificity
tn, fp, fn, tp = conf_mat.ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)

              precision    recall  f1-score   support

           0       0.94      0.97      0.95      1564
           1       0.97      0.94      0.95      1495

    accuracy                           0.95      3059
   macro avg       0.95      0.95      0.95      3059
weighted avg       0.95      0.95      0.95      3059

[[1515   49]
 [  94 1401]]
Accuracy: 0.9532526969597908
Precision: 0.9538927584063779
Recall: 0.9528969112728704
F1 Score: 0.953187682360237
ROC AUC: 0.9905526948310225
Specificity: 0.9686700767263428


In [65]:
# 获取特征重要性
feature_importances = best_rf.feature_importances_

# 排序特征
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 打印特征重要性排序
print(importance_df)
print(importance_df.head(10))

            Feature  Importance
109        Chronic5    0.074859
107        Chronic3    0.069524
93          Stress2    0.054349
92          Stress1    0.052591
94          Stress3    0.042601
..              ...         ...
57   MedCompliance1    0.001401
13          Smoking    0.001336
0         Residence    0.001321
62   MedCompliance6    0.001299
58   MedCompliance2    0.001290

[111 rows x 2 columns]
       Feature  Importance
109   Chronic5    0.074859
107   Chronic3    0.069524
93     Stress2    0.054349
92     Stress1    0.052591
94     Stress3    0.042601
91     EBS_SF7    0.041090
108   Chronic4    0.035902
85     EBS_SF1    0.026872
71   EQ_5D_5L5    0.026105
89     EBS_SF5    0.025626
